# 1D Heisenberg J1J2J3 model: DMRG

In this notebook, we use DMRG to calculate the ground state energies of the 1D Heisenberg J1J2J3 model with N=30 spins, J1 = 1.0, (J2, J3) = (0.0, 0.5), (0.2, 0.2), (0.2, 0.5), (0.5, 0.2)

In [1]:
from tenpy.models.spins import SpinModel
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg
from tenpy.models.lattice import Chain
from tenpy.networks.site import SpinHalfSite
from tenpy.models.model import CouplingMPOModel

import warnings
warnings.filterwarnings("ignore")

https://tenpy.readthedocs.io/en/latest/reference/tenpy.models.model.CouplingMPOModel.html

In [2]:
def define_model_params(N,j1,j2, j3, bc_MPSS, bc_xx):
    #bc_xx is either 'open' or 'periodic'
    #bc_MPSS is either 'finite' or 'infinite'
    model_params = {
                'conserve': None,
                'Lx': N,
                'J1': j1,
                'J2': j2,
                'J3': j3,
                'bc_MPS': bc_MPSS, 
                'bc_x': bc_xx, 
                }
    return model_params

In [3]:
class MyModel(CouplingMPOModel):     
    def init_sites(self, model_params):
        site = SpinHalfSite(conserve='None')
        return site
    
    def init_lattice(self, model_params):
        Lx =  model_params.get('Lx',10)
        bc_x = model_params.get('bc_x','open')
        bc_MPS = model_params.get('bc_MPS', 'finite')
        lattice = Chain(Lx, site=self.init_sites(model_params), bc_MPS=bc_MPS)
        return lattice
    def init_terms(self, model_params):
        J1 = model_params.get('J1', 1.)
        J2 = model_params.get('J2', 0.2)
        J3 = model_params.get('J3', 0.2)
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(J1, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J1, u1, 'Sy', u2, 'Sy', dx)
            self.add_coupling(J1, u1, 'Sx', u2, 'Sx', dx)
        for u1, u2, dx in self.lat.pairs['next_nearest_neighbors']:
            self.add_coupling(J2, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J2, u1, 'Sy', u2, 'Sy', dx)
            self.add_coupling(J2, u1, 'Sx', u2, 'Sx', dx)
        for u1, u2, dx in self.lat.pairs['next_next_nearest_neighbors']:
            self.add_coupling(J3, u1, 'Sz', u2, 'Sz', dx)
            self.add_coupling(J3, u1, 'Sy', u2, 'Sy', dx)
            self.add_coupling(J3, u1, 'Sx', u2, 'Sx', dx)


In [4]:
dmrg_params = {
    'mixer': True,  
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    'verbose': True,
}


In [5]:
#test_model = MyModel(model_params)
#test_model.init_lattice(model_params)
#test_model.lat.pairs['nearest_neighbors']

In [5]:
def run_dmrg(N, j1, j2, j3, bc_MPSS, bc_xx):
    model_params = define_model_params(N, j1, j2, j3, bc_MPSS, bc_xx)
    M = MyModel(model_params)
    lattice = M.init_lattice(model_params)
    initial_state = ['up'] * N # initial state
    psi = MPS.from_product_state(lattice.mps_sites(), initial_state, bc=lattice.bc_MPS)

    eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
    E,psi = eng.run() # the main work; modifies psi in place
    return E, psi

# N= 30

In [6]:
E30, psi30 = run_dmrg(30,1.,0.0, 0.5, 'finite', 'open')
print("ground state energy = ", E30)

ground state energy =  -15.890276367036542


In [7]:
E30, psi30 = run_dmrg(30,1.,0.2, 0.2, 'finite', 'open')
print("ground state energy = ", E30)

ground state energy =  -12.943044354682026


In [10]:
E30, psi30 = run_dmrg(30,1.,0.2, 0.5, 'finite', 'open')
print("ground state energy = ", E30)

ground state energy =  -14.640825798271765


In [8]:
E30, psi30 = run_dmrg(30,1.,0.5, 0.2, 'finite', 'open')
print("ground state energy = ", E30)

ground state energy =  -11.528738924487726


In [12]:
E30, psi30 = run_dmrg(30,1.,0.25, 0.25, 'finite', 'open')
print("ground state energy = ", E30)

ground state energy =  -12.910087313000062
